# Panic и recover

Система типов Go позволяет найти многие ошибки во время компиляции, но некоторые ошибки как выход за границу массива или разыменовывание требует проверки во время выполнения программы. Когда runtime языка Go находит такого рода ошибки, оно **паникует**(panic).

Когда происходит паника, выполнение программы останавливается, выполняются все отложенные функции горутины, в которой произошла паника, и программа падает с выводом сообщения. Это сообщение содержит **значение паники**(panic value), которое обычно является сообщением об ошибке, и **трейс стека**(stack trace - стек вызовов функций) каждой активной во время паники горутины. Обычно этого сообщения бывает достаточно для определения причины паники.

Не всегда паника исходит от runtime'а. Встроенная функция **panic** позволяет вызвать панику напрямую. На вход она принимает значение любого типа. Панику обычно вызывают в случаях, когда происходит что-то совсем "невозможное".

Хотя механизм паник похож на механизм исключений из других языков программирования, ситуации, в которых вызывается паника, совершенно иные. Т.к. из-за паники программа завершает выполнение, паника в основном используются как посмертный лог, который подскажет разработчику причину падения. "Нормальные" ошибки, которые являются ожидаемыми программой, такие как неправильный ввод или настройка, ошибка при чтении данных из файла, невозможность сетевого подключения и т.д. не должны рассматриваться как исключения: программа должна уметь работать с такими ошибками.

Как уже говорилось, после паники выполняются все отложенные функции горутины, в которой возникла паника:

In [2]:
import "fmt"

func f(n int) {
    fmt.Printf("f(%d)\n", n + 0/n) // Паника, когда n равно 0.
    defer fmt.Printf("defer %d\n", n)
    f(n - 1)
}

func main() {
    f(3)
}

main()

f(3)
f(2)
f(1)
defer 1
defer 2
defer 3


panic: runtime error: integer divide by zero

goroutine 67 [running]:
runtime/debug.Stack(0xc400000008, 0x7fb57a9cf450, 0xc42012e440)
	/usr/local/go/src/runtime/debug/stack.go:24 +0xa9
github.com/yunabe/lgo/core.(*resultCounter).recordResult(0xc42012e428, 0x7fb57a8e1160, 0x7fb57acf1470)
	/home/yunus/Documents/src/go/src/github.com/yunabe/lgo/core/core.go:95 +0xce
github.com/yunabe/lgo/core.(*resultCounter).recordResultInDefer(0xc42012e428)
	/home/yunus/Documents/src/go/src/github.com/yunabe/lgo/core/core.go:100 +0x3b
panic(0x7fb57a8e1160, 0x7fb57acf1470)
	/usr/local/go/src/runtime/panic.go:491 +0x294
github.com/yunabe/lgo/sess7b2274696d65223a313533343139343335383332373238353234347d/exec2.LgoExport_f(0x0)
	/home/yunus/Documents/src/go/src/github.com/yunabe/lgo/sess7b2274696d65223a313533343139343335383332373238353234347d/exec2/src.go:7 +0x1c0
github.com/yunabe/lgo/sess7b2274696d65223a313533343139343335383332373238353234347d/exec2.LgoExport_f(0x1)
	/home/yunus/Documents/src/go/src/github.

Также в Go возможно **восстановиться** от паники. Восстановление не приведёт к падению программы.